In [2]:
import os

BASE_DIR = 'stadium_dataset'
os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(f"{BASE_DIR}/images", exist_ok=True)
os.makedirs(f"{BASE_DIR}/annotations", exist_ok=True)

In [3]:
from PIL import Image, ImageDraw
import random

def generate_fan(team='hilal', action='sitting'):
    """
    Creates a cartoon fan as a colored stick figure.
    """
    img = Image.new('RGBA', (128, 128), (255, 255, 255, 0))
    draw = ImageDraw.Draw(img)

    # Team color
    color = (0, 0, 255) if team == 'hilal' else (255, 215, 0)

    # Head
    draw.ellipse((50, 10, 78, 38), fill=(255, 224, 189))

    # Body
    draw.rectangle((60, 38, 68, 90), fill=color)

    # Action variations
    if action == 'cheering':
        # Arms up
        draw.line((60, 40, 30, 20), fill=color, width=5)
        draw.line((68, 40, 98, 20), fill=color, width=5)
    elif action == 'throwing':
        draw.line((60, 40, 90, 35), fill=color, width=5)
        draw.line((68, 40, 70, 70), fill=color, width=5)
    elif action == 'fighting':
        draw.line((60, 40, 40, 50), fill=color, width=5)
        draw.line((68, 40, 85, 50), fill=color, width=5)
    else:  # Sitting
        draw.line((60, 40, 40, 60), fill=color, width=5)
        draw.line((68, 40, 90, 60), fill=color, width=5)

    return img


In [4]:
def create_stadium_background(filled=True):
    """
    Generates a basic stadium stand background.
    """
    img = Image.new('RGB', (512, 384), (150, 150, 150))
    draw = ImageDraw.Draw(img)

    # Seat rows
    for y in range(100, 350, 30):
        draw.rectangle((10, y, 502, y + 20), fill=(80, 80, 80))

    return img


In [5]:
import json
from tqdm import tqdm

annotations = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "fan", "supercategory": "person"}
    ]
}

fan_id = 1
image_id = 1

for i in tqdm(range(500)):
    bg = create_stadium_background(filled=True)
    fans = []
    num_fans = random.randint(3, 8)

    boxes = []
    for _ in range(num_fans):
        team = random.choice(['hilal', 'ittihad'])
        action = random.choice(['sitting', 'cheering', 'fighting', 'throwing'])

        fan_img = generate_fan(team, action)
        x = random.randint(20, 400)
        y = random.randint(120, 300)
        bg.paste(fan_img, (x, y), fan_img)

        # Save bbox for labeling
        boxes.append({
            "bbox": [x, y, 128, 128],
            "category_id": 1,
            "id": fan_id,
            "image_id": image_id,
            "attributes": {"team": team, "action": action}
        })

        fan_id += 1

    # Save image
    img_path = f"{BASE_DIR}/images/{image_id:04d}.png"
    bg.save(img_path)

    annotations["images"].append({
        "id": image_id,
        "file_name": f"{image_id:04d}.png",
        "width": 512,
        "height": 384
    })

    annotations["annotations"].extend(boxes)
    image_id += 1


100%|██████████| 500/500 [00:07<00:00, 63.02it/s] 


In [6]:
with open(f"{BASE_DIR}/annotations/labels.json", "w") as f:
    json.dump(annotations, f, indent=2)
